## Preliminaries

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data preview

In [15]:
df1 = pd.read_csv('CCXI.csv', index_col="Date", parse_dates=["Date"])
df2 = pd.read_csv('ESI.csv', index_col="Date", parse_dates=["Date"])
df = pd.concat([df1, df2], axis=1)

In [16]:
pio.templates.default = "plotly_white"

plot_template = dict(
    layout=go.Layout({
        "font_size": 18,
        "xaxis_title_font_size": 24,
        "yaxis_title_font_size": 24})
    )

fig = px.line(df1, labels=dict(
    Date="Date", value="CCXI"
    ))
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
    )
fig.show()

In [17]:
fig = px.line(df2, labels=dict(
    Date="Date", value="ESI"
    ))
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
    )
fig.show()

## Data normalization

In [18]:
target = "CCCI"
features = ["CCCI"]
long_term = True

df_train = df.loc[:"2021-03-01"] if long_term else df.loc[:"2021-12-01"]

target_mean = df_train[target].mean()
target_std = df_train[target].std()

for c in df_train.columns:
    mu = df_train.loc[:,c].mean()
    std = df_train.loc[:,c].std()
    
    df.loc[:,c] = (df.loc[:,c] - mu) / std
    
df_train = df.loc[:"2021-03-01"] if long_term else df.loc[:"2021-12-01"]
df_test  = df.loc["2021-04-01":] if long_term else df.loc["2022-01-01":]
test_start = "2021-04-01" if long_term else "2022-01-01"

## Dataset

In [19]:
class CCXI(Dataset):
    def __init__(self, target, features, seq_length = 5, train = True):
        if train:
            dataset = df_train
        else:
            dataset = df_test
        self.y = torch.tensor(dataset[target].values).float()
        self.X = torch.tensor(dataset[features].values).float()
        self.seq_length = seq_length
        
        if train:
            self.past = torch.zeros_like(self.X[0]).repeat(seq_length, 1)
        else:
            self.past = torch.tensor(df_train.iloc[(df_train.shape[0]-seq_length):,].loc[:,features].values).float()
        
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, i):
        if i >= self.seq_length:
            i_start = i - self.seq_length
            X = self.X[i_start:i, :]
        else:
            pad = self.past[i:]
            X = self.X[:i, :]
            X = torch.cat((pad, X), 0)
        return X, self.y[i]

In [20]:
seq_length = 12
batch_size = 8

train_dataset = CCXI(target, features, seq_length, train = True)
test_dataset = CCXI(target, features, seq_length, train = False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

X, y = next(iter(train_loader))
print("Features shape:", X.shape)
print("Target shape:", y.shape)

Features shape: torch.Size([8, 12, 1])
Target shape: torch.Size([8])


## LSTM model

In [21]:
class LSTM(nn.Module):
    def __init__(self, num_features, hidden_units):
        super().__init__()
        self.num_features = num_features
        self.hidden_units = hidden_units
        self.num_layers = 12
        
        self.lstm = nn.LSTM(
            input_size = num_features,
            hidden_size = hidden_units,
            batch_first = True,
            num_layers = self.num_layers)
        
        self.linear = nn.Linear(in_features = self.hidden_units, out_features = 1)
        
    def forward(self, x):
        batch_size = x.shape[0]
        
        _, (hn, _) = self.lstm(x)
        out = self.linear(hn[0]).flatten()
        
        return out
    
num_hidden_units = 12
model = LSTM(num_features = len(features), hidden_units = num_hidden_units).to(device)

## Loss & Optimizer

In [22]:
learning_rate = 5e-4

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train

In [23]:
num_epochs = 500
num_batch = len(train_loader)

train_losses = []
test_losses = []

for epoch in range(num_epochs):
    total_loss = 0
    # train set
    for X, y in train_loader:
        optimizer.zero_grad()
        
        output = model(X.to(device))
        loss = loss_function(output, y.to(device))
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    train_avg = total_loss / num_batch * 1000
    # test set
    with torch.no_grad():
        test_loss = 0
        
        for X, y in test_loader:
            output = model(X.to(device))
            test_loss += loss_function(output, y.to(device)).item()
    
    test_avg = test_loss / num_batch * 1000
    train_losses.append(train_avg)
    test_losses.append(test_avg)
    if (epoch + 1) % 50 == 0:
        print(f"({epoch+1:3d}/{num_epochs}) Train loss: {train_avg:.4f}, Test loss: {test_avg:.4f}")  

( 50/500) Train loss: 100.1852, Test loss: 16.0832
(100/500) Train loss: 64.1550, Test loss: 8.9170
(150/500) Train loss: 53.0439, Test loss: 6.6542
(200/500) Train loss: 49.2343, Test loss: 6.9261
(250/500) Train loss: 53.5089, Test loss: 7.2989
(300/500) Train loss: 45.3802, Test loss: 6.2142
(350/500) Train loss: 43.3448, Test loss: 5.4266
(400/500) Train loss: 43.4919, Test loss: 3.7219
(450/500) Train loss: 35.9813, Test loss: 2.8630
(500/500) Train loss: 33.0021, Test loss: 3.1414


In [24]:
df_loss = pd.DataFrame({"Train":train_losses, "Test":test_losses}, index=list(range(num_epochs)))
fig = px.line(df_loss[50:], labels=dict(Epoch="Epoch", value="Loss"))
fig.update_layout(
  template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
    )
fig.show()

## Prediction

In [41]:
def predict(data_loader, model):
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_hat = model(X.to(device)).cpu()
            output = torch.cat((output, y_hat), 0)
    return output

train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

train_hat = predict(train_eval_loader, model).numpy()
test_hat = predict(test_loader, model).numpy()
df["forecast"] = np.concatenate((train_hat, test_hat), 0)

df_out = df.loc[:, [target, "forecast"]].copy()

for c in df_out.columns:
    df_out[c] = df_out[c] * target_std + target_mean

In [43]:
new_test_hat = torch.tensor([]).numpy()
for i in range(len(test_hat)):
    if i >= seq_length:
        i_start = i - seq_length-1
        X = new_test_hat[i_start:i]
    else:
        pad = train_hat[(len(train_hat)-seq_length + i):]
        X = new_test_hat[:i]
        X = np.concatenate((pad, X), 0)
    X = X.reshape((1,12,1))
    y_hat = model(torch.tensor(X).to(device)).cpu().detach().numpy()
    new_test_hat = np.concatenate((new_test_hat, y_hat), 0)

df["forecast2"] = np.concatenate((train_hat, new_test_hat), 0)
df_out2 = df.loc[:, [target, "forecast2"]].copy()

for c in df_out2.columns:
    df_out2[c] = df_out2[c] * target_std + target_mean

## Plot

In [14]:
fig = px.line(df_out, labels=dict(Date="Date", value=target))
fig.add_vline(x=test_start, line_width=4, line_dash="dash")
fig.update_layout(
    template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()

In [44]:
fig = px.line(df_out2, labels=dict(Date="Date", value=target))
fig.add_vline(x=test_start, line_width=4, line_dash="dash")
fig.update_layout(
    template=plot_template, legend=dict(orientation='h', y=1.02, title_text="")
)
fig.show()

In [37]:
MSE = ((df_out[target] - df_out["forecast"])**2).mean()
MAE = (abs(df_out[target] - df_out["forecast"])).mean()
MAPE = (abs(df_out[target] - df_out["forecast"])/df_out[target]).mean() * 100

In [38]:
print("MSE:",MSE)
print("MAE:",MAE)
print("MAPE:",MAPE)

MSE: 0.029987718929089094
MAE: 0.1275004993785508
MAPE: 0.12694043638575347
